In [ ]:
# from https://www.kaggle.com/takeshikobayashi/mnist-cnn-keras-with-optuna-visualization
# from https://machinelearningapplied.com/hyperparameter-search-with-optuna-part-3-keras-cnn-classification-and-ensembling/

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os


# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [54]:
#import dataset
import pandas as pd
import numpy as np

import optuna
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Conv1D, GlobalMaxPooling1D 

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
 
train = pd.read_csv('D:/FH_Dortmund/Projekt/Projektarbeit_2/time_series/dl-4-tsc-master/dataset/archives/UCRArchive_2018/BME/BME_TRAIN.tsv', sep='\t', header=None) 
test = pd.read_csv('D:/FH_Dortmund/Projekt/Projektarbeit_2/time_series/dl-4-tsc-master/dataset/archives/UCRArchive_2018/BME/BME_TEST.tsv', sep='\t', header=None)
print('The size of the train data:' + str(train.shape))
print('The size of the test data:' + str(test.shape))

The size of the train data:(30, 129)
The size of the test data:(150, 129)


In [91]:
# from read_dataset
y_train = train.values[:, 0]
y_test = test.values[:, 0]

X_train = train.drop(columns=[0])
X_test = test.drop(columns=[0])

X_train.columns = range(X_train.shape[1])
X_test.columns = range(X_test.shape[1])

X_train = X_train.values
X_test = X_test.values

# print("X_train is: ", X_train)
# print("X_train shape is: ", X_train.shape)
# print("X_test shape is: ", X_test.shape)
# print("y_train shape is: ", y_train)
# print("y_test shape is: ", y_test)

In [38]:
# Warum müssen Sie hier Daten in float-type umwandeln?

# transform float type

#X_train = (train.iloc[:,1:].values).astype('float32') 
#y_train = train.iloc[:,0].values
#X_test = (test.iloc[:,1:].values).astype('float32')


In [59]:
#train_test_split again
from sklearn.model_selection import train_test_split
 
X_train2, X_test2, y_train2, y_test2 = train_test_split(X_train,
                                                        y_train, 
                                                        test_size = 0.2,
                                                        train_size = 0.8) #教師データ少なくなるのが怖いので4:1で分割
                                                       # stratify = y_train)
print(f'X_train2 の長さ: {len(X_train2)}')
print(f'X_test2 の長さ: {len(X_test2)}')


X_train2 の長さ: 24
X_test2 の長さ: 6


In [60]:
#reshape data 
# img_rows, img_cols = 128, 128
# num_classes = 3

X_train = X_train2
X_test = X_test2


y_train= y_train2

In [61]:
X_train.shape

(24, 128)

In [62]:
import optuna
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv1D, GlobalAveragePooling1D, BatchNormalization

# img_rows, img_cols = 28, 28
num_classes = 3

# Das machen ich nichts.
# define the CNN model
def create_model(num_layer, mid_units, num_filters, dropout_rate):
    
    model = Sequential()
    model.add(Conv1D(filters=num_filters[0], kernel_size=8,
                 activation = "relu",
                 padding="same",
                 input_shape = (23, 0)))
    model.add(BatchNormalization())
    for i in range(1,num_layer):
        model.add(Conv1D(filters=num_filters[i], kernel_size=5, padding="same", activation="relu"))
    
    model.add(GlobalAveragePooling1D())
    model.add(Dropout(dropout_rate[0]))
 #  model.add(Flatten())
    model.add(Dense(mid_units))
    model.add(Dropout(dropout_rate[1]))
    model.add(Dense(num_classes, activation='softmax'))
    
    return model

In [89]:
def objective(trial):
    print("Optimize Start")
    
    #clear_session
    keras.backend.clear_session()
    
    #number of the convolution layer
    num_layer = trial.suggest_int("num_layer", 2, 4)
    
    #number of the each convolution layer filter
    num_filters = trial.suggest_categorical('num_filters', [16, 32, 64, 128])
    
    #kernel_size
    kernel_size = trial.suggest_int('kernel_size', 3, 8)
    
    #activation = trial.suggest_categorical("activation", ["relu", "sigmoid"])
    
    #batch_size
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 96, 128])
    
    #drop_out
    drop_out = trial.suggest_discrete_uniform('drop_out', 0.05, 0.5, 0.05)
    
    dict_params = {'num_layer':num_layer,
                   'num_filters':num_filters,
                   'kernel_size':kernel_size,
                   'batch_size':batch_size,
                   'drop_out':drop_out}
    
    
    #create cnn model
    # model = create_model(num_layer, mid_units, num_filters, dropout_rate,)
    # ------------------------------------------------------------------
    # start of cnn coding   
    self.input_shape = input_shape
    input_tensor = Input(shape=self.input_shape)
         
    # 1st cnn block
    x = BatchNormalization()(input_tensor)
    x = Activation('relu')(x)
    x = Conv1D(filters=dict_params['num_filters'],
               kernel_size=dict_params['kernel_size'], 
               strides=1, padding='same')(x)
    #x = MaxPooling2D()(x)
    x = Dropout(dict_params['drop_out'])(x)
         
    # additional cnn blocks
    for iblock in range(dict_params['num_cnn_blocks'] - 1):
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = Conv1D(filters=dict_params['num_filters'],
                   kernel_size=dict_params['kernel_size'],
                   strides=1, padding='same')(x)
        x = Dropout(dict_params['drop_out'])(x)
                             

    x = Dropout(dict_params['drop_out'])(x)
    x = GlobalAveragePooling1D(x)
  #  x = Dense(dict_params['num_dense_nodes']//dict_params['dense_nodes_divisor'], 
  #            activation='relu')(x)
    output_tensor = Dense(self.number_of_classes, activation='softmax')(x)
    
    #------------------------------------------------------------------
    
    #compile
    model = Model(inputs=input_tensor, outputs=output_tensor)
    opt = Adam(lr=0.00025)  # default = 0.001
    model.compile(optimizer=opt,
                  loss="categorical_crossentropy",
                  metrics=["accuracy"])
    
    history = model.fit(X_train, y_train, verbose=0, epochs=20, batch_size=128, validation_split=0.1)
    
    scores = model.evaluate(X_train, y_train)
    print('accuracy={}'.format(*scores))
    
    #検証用データに対する正答率が最大となるハイパーパラメータを求める
    #return 1 - history.history["val_acc"][-1]
    return 1 - history.history["val_acc"][-1]

In [90]:

study = optuna.create_study()
study.optimize(objective, n_trials=2)

[I 2022-03-16 13:50:27,821] A new study created in memory with name: no-name-5c319f58-1181-482c-b8fb-27a4d4149863
[W 2022-03-16 13:50:27,826] Trial 0 failed because of the following error: NameError("name 'input_shape' is not defined")
Traceback (most recent call last):
  File "D:\Anaconda\lib\site-packages\optuna\study\_optimize.py", line 213, in _run_trial
    value_or_values = func(trial)
  File "D:\TMP/ipykernel_9308/1806945611.py", line 48, in objective
    self.input_shape = input_shape
NameError: name 'input_shape' is not defined


Optimize Start


NameError: name 'input_shape' is not defined

In [ ]:
plt.plot([t.value for t in study.trials])

In [ ]:
optuna.visualization.plot_contour(study)

In [ ]:
# パラメータ多すぎてよくわからんので聞いているパラメータを選びます
optuna.visualization.plot_param_importances(study)

In [ ]:
optuna.visualization.plot_contour(study, params=["dropout_rate1","num_filter_0","num_layer","dropout_rate0","num_filter_1"])

In [ ]:
study.best_params

In [ ]:
optuna.visualization.plot_optimization_history(study)

In [ ]:
optuna.visualization.plot_optimization_history(study)

In [ ]:
optuna.visualization.plot_intermediate_values(study)

In [ ]:
optuna.visualization.plot_parallel_coordinate(study)

In [ ]:
optuna.visualization.plot_contour(study, params=["num_filter_0", "num_filter_1","num_filter_2","num_filter_3","num_filter_4"])

In [ ]:
optuna.visualization.plot_slice(study)